In [1]:
import tensorflow as tf
import keras
from keras.utils import plot_model
from keras.layers import Conv2D, MaxPool2D ,Input ,Conv2DTranspose,Concatenate,UpSampling2D,Cropping2D
from keras import Sequential,Model
import numpy as np

Using TensorFlow backend.


In [20]:
class UNet():
    def __init__(self, input_dims=None):
         self.img_shape = (input_dims, input_dims, 1)
    
   
   
    def create_model(self, filters=64, ks = (3,3),kI='he_normal', pad='valid',activation='relu'):  # by default padding is valid but here i specify so i can remember which padding to use
        # down sampling
        img = Input(shape=self.img_shape)
        x = img
        print(x)
        conv1 = Conv2D(filters,kernel_size=ks,kernel_initializer=kI, padding=pad,activation=activation)(x)
        conv1 = Conv2D(filters,kernel_size=ks,kernel_initializer=kI, padding=pad,activation=activation)(conv1)#
        max_pool1 = MaxPool2D((2,2))(conv1)
        print('Max pool 1:',max_pool1.shape)
        
        conv2 = Conv2D(filters*2,kernel_size=ks,kernel_initializer=kI, padding=pad,activation=activation)(max_pool1)
        conv2 = Conv2D(filters*2,kernel_size=ks,kernel_initializer=kI, padding=pad,activation=activation)(conv2)#
        max_pool2 = MaxPool2D((2,2))(conv2)
        print('Max pool 2:',max_pool2.shape)
        
        conv3 = Conv2D(filters*4,kernel_size=ks,kernel_initializer=kI, padding=pad,activation=activation)(max_pool2)
        conv3 = Conv2D(filters*4,kernel_size=ks,kernel_initializer=kI, padding=pad,activation=activation)(conv3) 
        max_pool3 = MaxPool2D((2,2))(conv3)
        print('Max pool 3:',max_pool3.shape)
        
        conv4 = Conv2D(filters*8,kernel_size=ks,kernel_initializer=kI, padding=pad,activation=activation)(max_pool3)
        conv4 = Conv2D(filters*8,kernel_size=ks,kernel_initializer=kI, padding=pad,activation=activation)(conv4)  #
        max_pool4 = MaxPool2D((2,2))(conv4)
        print('Max pool 4:',max_pool4.shape)
        print('conv 4',conv4.shape)
        
        conv5 = Conv2D(filters*16,kernel_size=ks,kernel_initializer=kI, padding=pad,activation=activation)(max_pool4)
        conv5 = Conv2D(filters*16,kernel_size=ks,kernel_initializer=kI, padding=pad,activation=activation)(conv5)
        print('Down sampling last layers output shape:',conv5.shape)
       
#         Upsampling 
        
        # upconv1 

        up_conv1 = UpSampling2D((2,2))(conv5)
        print(up_conv1.shape)
        
        cropping_size = conv4.shape[1] - up_conv1.shape[1]
        
        crop = Cropping2D(cropping=cropping_size//2)(conv4)
        print('cropped',crop.shape)
        merged = Concatenate()([crop, up_conv1])

        up_conv_conv1 = Conv2D(filters*8,kernel_size=ks,kernel_initializer=kI, padding=pad,activation=activation)(merged)
        up_conv_conv1= Conv2D(filters*8,kernel_size=ks,kernel_initializer=kI, padding=pad,activation=activation)(up_conv_conv1)

        print('Up-conv 1:',up_conv_conv1.shape)
        
        
        
        # upconv2 
        
        
        up_conv2 = UpSampling2D((2,2))(up_conv_conv1)
        print(up_conv2)
        cropping = crop_size(conv3,up_conv2)
        print(cropping.shape)
        
        cropping_size = conv3.shape[1] - up_conv2.shape[1]
        
        crop = Cropping2D(cropping=cropping_size//2)(conv3)
        print(crop.shape)
        merged = Concatenate()([up_conv2, crop])
#         print(merged.shape)
     
        up_conv_conv2 = Conv2D(filters*4,kernel_size=ks,kernel_initializer=kI, padding=pad,activation=activation)(merged)
        up_conv_conv2 = Conv2D(filters*4,kernel_size=ks,kernel_initializer=kI, padding=pad,activation=activation)(up_conv_conv2)

        print('Up_conv 2:',up_conv_conv2.shape)
        
        
        # upconv3
        up_conv3 = UpSampling2D((2,2))(up_conv_conv2)
       
        
        cropping_size = conv2.shape[1] - up_conv3.shape[1]
        
        crop = Cropping2D(cropping=cropping_size//2)(conv2)
        merged = Concatenate()([up_conv3, crop])
        print('Conv 3 merger',merged.shape)
     
        up_conv_conv3 = Conv2D(filters*2,kernel_size=ks,kernel_initializer=kI, padding=pad,activation=activation)(merged)
        up_conv_conv3 = Conv2D(filters*2,kernel_size=ks,kernel_initializer=kI, padding=pad,activation=activation)(up_conv_conv3)

        print('Up-conv 3:',up_conv_conv3.shape)
        
         # upconv4
        
        up_conv4 = UpSampling2D((2,2))(up_conv_conv3)
       
        
        cropping_size = conv1.shape[1] - up_conv4.shape[1]
        
        crop = Cropping2D(cropping=cropping_size//2)(conv1)
        merged = Concatenate()([up_conv4, crop])

     
        up_conv_conv4 = Conv2D(filters*1,kernel_size=ks,kernel_initializer=kI, padding=pad,activation=activation)(merged)
        up_conv_conv4 = Conv2D(filters*1,kernel_size=ks,kernel_initializer=kI, padding=pad,activation=activation)(up_conv_conv4)

        print('Up-conv 4:',up_conv_conv4.shape)
        
        out = Conv2D(filters=1, kernel_size=(1,1), padding='same', activation='sigmoid')(up_conv_conv4)
        model = Model(inputs=img,outputs=out)
        return model

In [21]:
if __name__ == "__main__":
    img = np.random.rand(572,572,1) * 255
    img_shape = img.shape[0]
    net = UNet(img_shape)
    model = net.create_model()
    model.summary()
    plot_model(model,to_file='model.png',show_shapes=True,rankdir='LB')
    

Tensor("input_8:0", shape=(None, 572, 572, 1), dtype=float32)
Max pool 1: (None, 284, 284, 64)
Max pool 2: (None, 140, 140, 128)
Max pool 3: (None, 68, 68, 256)
Max pool 4: (None, 32, 32, 512)
conv 4 (None, 64, 64, 512)
Down sampling last layers output shape: (None, 28, 28, 1024)
(None, 56, 56, 1024)
cropped (None, 56, 56, 512)
Up-conv 1: (None, 52, 52, 512)
Tensor("up_sampling2d_30/ResizeNearestNeighbor:0", shape=(None, 104, 104, 512), dtype=float32)
(None, 104, 104, 256)
(None, 104, 104, 256)
Up_conv 2: (None, 100, 100, 256)
Conv 3 merger (None, 200, 200, 384)
Up-conv 3: (None, 196, 196, 128)
Up-conv 4: (None, 388, 388, 64)
Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 572, 572, 1)  0                                            
____________________________________________________